In [28]:
pip install pandas  

In [29]:
pip install pmdarima

In [30]:
pip install pyramid-arima

In [31]:
from pandas.plotting import lag_plot,autocorrelation_plot
from pandas import read_csv
from pandas import datetime
from pandas import DataFrame
from matplotlib import pyplot as plt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.ar_model import AutoReg
from sklearn.externals import joblib
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMAResults
from itertools import zip_longest
from pyramid.arima import auto_arima
import matplotlib as mpl
import seaborn as sns
import statsmodels.api as sm
import os.path
import pmdarima as pm
import csv


total_MSE = 0


In [32]:
def parser(x):
	return datetime.strptime('190'+x, '%Y-%m')

In [33]:
# monkey patch around bug in ARIMA class
def __getnewargs__(self):
	return ((self.endog),(self.k_lags, self.k_diff, self.k_ma))
ARIMA.__getnewargs__ = __getnewargs__

In [34]:
# 讀取 CSV File
import pandas as pd # 引用套件並縮寫為 pd  
!ls
#base_url = "https://drive.google.com/file/d/15_QRNkXUkvRn1KU6VpcpGhqsH9LNfmbD/view?usp=sharing"
#df = pd.read_csv(base_url, encoding='big5')
df = pd.read_csv("dataset.csv")
df = df.fillna(0)
print(df)  

 106060018_HW1.csv
 Afghanistan.model
 Albania.model
 Algeria.model
 Andorra.model
 Angola.model
 Anguilla.model
 Antigua_and_Barbuda.model
 Argentina.model
 Armenia.model
 Aruba.model
 Australia.model
 Austria.model
 Azerbaijan.model
 Bahamas.model
 Bahrain.model
 Bangladesh.model
 Barbados.model
 Belarus.model
 Belgium.model
 Belize.model
 Benin.model
 Bermuda.model
 Bhutan.model
 Bolivia.model
'Bonaire, Saint Eustatius and Saba.model'
 Bosnia_and_Herzegovina.model
 Botswana.model
 Brazil.model
 British_Virgin_Islands.model
 Brunei_Darussalam.model
 Bulgaria.model
 Burkina_Faso.model
 Burundi.model
 Cambodia.model
 Cameroon.model
 Canada.model
 Cape_Verde.model
 Cases_on_an_international_conveyance_Japan.model
 Cayman_Islands.model
 Central_African_Republic.model
 Chad.model
 Chile.model
 China.model
 Colombia.model
 Comoros.model
 Congo.model
 Costa_Rica.model
 Cote_dIvoire.model
 Croatia.model
 Cuba.model
 Curaçao.model
 Cyprus.model
 Czechia.model
 dataset.csv
 Democratic_Republic

In [42]:
n = 211
lists = [[] for i in range(n)]
lists[0] = [" ", '10/9', '10/10', '10/11', '10/12', '10/13', '10/14', '10/15']
print(lists)

df_grouped = df.groupby('countriesAndTerritories')
#AFG_group = df_grouped.get_group('Afghanistan')
#print(AFG_group)

country_num = 1

for name,group in df_grouped:
  COUNTRY = str(name)
  print(COUNTRY)
  MODEL_PATH = str(name) + ".pkl"
  #MODEL_PATH = "./drive/My Drive/Machine Learning/" + str(name) + ".pkl"
  if os.path.exists(MODEL_PATH):
    model = ARIMAResults.load(MODEL_PATH)
  AFG_group = df_grouped.get_group(COUNTRY)
  AFG_group.dateRep = pd.to_datetime({'year': AFG_group['year'],
                   'month': AFG_group['month'],
                   'day': AFG_group['day']}, 
                   format='%d/%m/%Y')
  AFG_group.set_index('dateRep', inplace=True)
  AFG_group_cases = AFG_group.filter(items=['dateRep', 'cases'])
  AFG_group_cases = AFG_group_cases.iloc[::-1]

  X = AFG_group_cases.values
  #size = int(len(X) * 0.66)
  #size = int(len(X) - 7)
  #train, test = X[0:size], X[size:len(X)]
  history = [x for x in X]
  for i in range(len(history)):
    if history[i] < 0:
      history[i] = 0
  predictions = list()
  #for t in range(len(test)):
  model = ARIMA(history, order=(4,1,0))
  model_fit = model.fit(disp=0)
  output = model_fit.forecast(steps=7)[0]
  day = 0
  lists[country_num].append(COUNTRY)
  for yhat in output:
    if yhat<0 :
      yhat = 0
    yhat_round = int(round(yhat))
    print('Day %d: %d' % (day, yhat_round))
    lists[country_num].append(yhat_round)
    history.append(yhat_round)
    day += 1
  country_num += 1
  '''
  yhat = output[0]
  if yhat<0 :
    yhat = 0
  predictions.append(yhat)
  obs = test[t]
  history.append(obs)
  print('predicted=%f, expected=%f' % (yhat, obs))
  
  #error = mean_squared_error(test, predictions)
  print('Test MSE: %.3f' % error)
  total_MSE += error
  # plot
  plt.plot(test)
  plt.plot(predictions, color='red')
  plt.show()
  '''
  # save model
  model_fit.save(MODEL_PATH)
  # load model
  loaded = ARIMAResults.load(MODEL_PATH)

#print("total MSE: %.3f" % total_MSE)
rows = zip_longest(*lists, fillvalue = '')
with open("106060018_HW1.csv", "w") as f:
  writer = csv.writer(f)
  for row in rows:
    writer.writerow(row)


[[' ', '10/9', '10/10', '10/11', '10/12', '10/13', '10/14', '10/15'], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
Afghanistan
Day 0: 64
Day 1: 84
Day 2: 86
Day 3: 71
Day 4: 75
Day 5: 78
Day 6: 80
Albania


/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


Day 0: 159
Day 1: 161
Day 2: 159
Day 3: 161
Day 4: 162
Day 5: 162
Day 6: 163
Algeria
Day 0: 122
Day 1: 120
Day 2: 120
Day 3: 120
Day 4: 120
Day 5: 120
Day 6: 120
Andorra
Day 0: 41
Day 1: 116
Day 2: 145
Day 3: 63
Day 4: 153
Day 5: 75
Day 6: 129
Angola
Day 0: 225
Day 1: 45
Day 2: 233
Day 3: 93
Day 4: 178
Day 5: 109
Day 6: 175
Anguilla
Day 0: 0
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
Antigua_and_Barbuda
Day 0: 0
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
Argentina
Day 0: 14008
Day 1: 12400
Day 2: 12691
Day 3: 13653
Day 4: 14589
Day 5: 14250
Day 6: 13783
Armenia
Day 0: 598
Day 1: 558
Day 2: 485
Day 3: 493
Day 4: 546
Day 5: 556
Day 6: 551
Aruba
Day 0: 16
Day 1: 11
Day 2: 10
Day 3: 12
Day 4: 13
Day 5: 13
Day 6: 12
Australia
Day 0: 18
Day 1: 16
Day 2: 16
Day 3: 15
Day 4: 17
Day 5: 16
Day 6: 16
Austria
Day 0: 817
Day 1: 826
Day 2: 899
Day 3: 818
Day 4: 875
Day 5: 853
Day 6: 862
Azerbaijan
Day 0: 161
Day 1: 166
Day 2: 174
Day 3: 172
Day 4: 172
Day 5: 174
Day 6: 174
Baha

/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:548: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/base/model.py:568: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


Day 0: 3146
Day 1: 438
Day 2: 3070
Day 3: 1657
Day 4: 2538
Day 5: 3615
Day 6: 922
Russia
Day 0: 11482
Day 1: 11497
Day 2: 11615
Day 3: 11619
Day 4: 11714
Day 5: 11746
Day 6: 11803
Rwanda
Day 0: 9
Day 1: 11
Day 2: 8
Day 3: 8
Day 4: 8
Day 5: 8
Day 6: 9
Saint_Kitts_and_Nevis
Day 0: 0
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
Saint_Lucia
Day 0: 0
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
Saint_Vincent_and_the_Grenadines
Day 0: 0
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
San_Marino
Day 0: 5
Day 1: 5
Day 2: 3
Day 3: 8
Day 4: 4
Day 5: 5
Day 6: 5
Sao_Tome_and_Principe
Day 0: 1
Day 1: 0
Day 2: 0
Day 3: 0
Day 4: 0
Day 5: 0
Day 6: 0
Saudi_Arabia
Day 0: 464
Day 1: 489
Day 2: 490
Day 3: 489
Day 4: 497
Day 5: 499
Day 6: 501
Senegal
Day 0: 27
Day 1: 27
Day 2: 27
Day 3: 28
Day 4: 28
Day 5: 28
Day 6: 28
Serbia
Day 0: 123
Day 1: 130
Day 2: 121
Day 3: 127
Day 4: 126
Day 5: 127
Day 6: 128
Seychelles
Day 0: 1
Day 1: 1
Day 2: 1
Day 3: 0
Day 4: 1
Day 5: 1
Day 6: 1
Sierra_L

In [36]:
print(lists[209])

['Zambia', 70, 65, 63, 67, 65, 66, 67]


In [48]:
# load model
loaded = ARIMAResults.load('Russia.pkl')
output = loaded.forecast(steps=7)[0]
day = 0
for yhat in output:
  if yhat<0 :
    yhat = 0
  yhat_round = int(round(yhat))
  print('Day %d: %d' % (day, yhat_round))
  day += 1

Day 0: 11482
Day 1: 11497
Day 2: 11615
Day 3: 11619
Day 4: 11714
Day 5: 11746
Day 6: 11803
